<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/2_uygulama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# DOSYA ADI: egitim.py
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle # Kayıt işlemleri için
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, recall_score, cohen_kappa_score

# --- 1. ADIM: TEKNOFEST ÜNİVERSİTE VERİSİ OLUŞTURMA ---
print(">> Veri üretiliyor (200.000 Adet)...")

def veri_uret(n):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        # Genomik (DNA) ve Proteomik (Amino Asit) Komşuluk (11 birim)
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Sayısal Özellikler (Risk, MAF, Korunmuşluk, Hidro, Polarite, Ağırlık)
        risk = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        cons = np.random.uniform(0, 10)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        # Etiketleme Kuralı (Simülasyon)
        score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
        score += np.random.normal(0, 0.05)
        label = 1 if score > 0.65 else 0

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    return dna_seqs, prot_seqs, np.array(bio_feats), np.array(labels)

dna_data, prot_data, num_data, y = veri_uret(200000)

# --- 2. ADIM: VERİYİ İŞLEME VE HAZIRLAMA ---
# DNA Tokenizer
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_data)
X_dna = pad_sequences(dna_tok.texts_to_sequences(dna_data), maxlen=11, padding='post')

# Protein Tokenizer
prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_data)
X_prot = pad_sequences(prot_tok.texts_to_sequences(prot_data), maxlen=11, padding='post')

# Eğitim/Test Ayrımı
X_dna_tr, X_dna_ts, X_prot_tr, X_prot_ts, X_num_tr, X_num_ts, y_tr, y_ts = train_test_split(
    X_dna, X_prot, num_data, y, test_size=0.2, random_state=42
)

# --- 3. ADIM: MODEL MİMARİSİ (3 GİRDİLİ) ---
# Giriş 1: DNA
in_dna = Input(shape=(11,))
emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_dna))

# Giriş 2: Protein
in_prot = Input(shape=(11,))
emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_prot))

# Giriş 3: Sayısal Veriler
in_num = Input(shape=(6,))
x3 = BatchNormalization()(Dense(16, activation='relu')(in_num))

# Birleştirme
merged = Concatenate()([x1, x2, x3])
z = Dropout(0.4)(Dense(64, activation='relu')(merged))
out = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[in_dna, in_prot, in_num], outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 4. ADIM: EĞİTİM ---
print(">> Model Eğitiliyor...")
model.fit([X_dna_tr, X_prot_tr, X_num_tr], y_tr, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

# --- 5. ADIM: RAPORLAMA ---
print("\n>> Performans Hesaplanıyor...")
preds = (model.predict([X_dna_ts, X_prot_ts, X_num_ts]) > 0.5).astype(int)

print(f"F1 Skoru : {f1_score(y_ts, preds):.4f}")
print(f"Kappa    : {cohen_kappa_score(y_ts, preds):.4f}")
print(f"Recall   : {recall_score(y_ts, preds):.4f}")

# --- 6. ADIM: KAYDETME (EN ÖNEMLİ KISIM) ---
print("\n>> Dosyalar kaydediliyor (Bir sonraki kod için)...")

# Modeli kaydet (.h5)
model.save('model/teknofest_beyni.h5')

# Tokenizer'ları kaydet (Harf çeviriciler)
with open('model/dna_sozlugu.pickle', 'wb') as f:
    pickle.dump(dna_tok, f)

with open('model/prot_sozlugu.pickle', 'wb') as f:
    pickle.dump(prot_tok, f)

print("✅ BİTTİ! 'teknofest_beyni.h5', 'dna_sozlugu.pickle' ve 'prot_sozlugu.pickle' oluştu.")

>> Veri üretiliyor (200.000 Adet)...
>> Model Eğitiliyor...
Epoch 1/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7934 - loss: 0.4258 - val_accuracy: 0.8948 - val_loss: 0.2369
Epoch 2/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8844 - loss: 0.2609 - val_accuracy: 0.8869 - val_loss: 0.2551
Epoch 3/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8859 - loss: 0.2567 - val_accuracy: 0.8933 - val_loss: 0.2349
Epoch 4/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8861 - loss: 0.2538 - val_accuracy: 0.9004 - val_loss: 0.2287
Epoch 5/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8866 - loss: 0.2554 - val_accuracy: 0.8976 - val_loss: 0.2343
Epoch 6/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8860 - loss: 0.2551 - val_accuracy: 0.8959 - val_loss: 0.2326
Epoch 7/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8869 - loss: 0.2544 - val_accuracy: 0.8984 - val_loss: 0.2300
Epoch 8/10
2250/2250 ━━━━━━━

F1 Skoru : 0.8288
Kappa    : 0.7533
Recall   : 0.8247

>> Dosyalar kaydediliyor (Bir sonraki kod için)...
✅ BİTTİ! 'teknofest_beyni.h5', 'dna_sozlugu.pickle' ve 'prot_sozlugu.pickle' oluştu.


In [28]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. MODELİ VE SÖZLÜKLERİ YÜKLE
print(">> Model ve sözlükler yükleniyor...")
model = load_model('model/teknofest_beyni.h5')

with open('model/dna_sozlugu.pickle', 'rb') as f:
    dna_tok = pickle.load(f)

with open('model/prot_sozlugu.pickle', 'rb') as f:
    prot_tok = pickle.load(f)

# 2. GERÇEK CEVAPLARIN OLDUĞU DOSYAYI YÜKLE
# Önceki eğitimde kaydettiğimiz dosyayı okuyoruz
print(">> Test sonuçları dosyası okunuyor...")
try:
    df = pd.read_csv('teknofest_univ_test_sonuclari.txt', sep='\t')
except FileNotFoundError:
    print("HATA: 'teknofest_univ_test_sonuclari.txt' dosyası bulunamadı!")
    print("Lütfen önce egitim.py kodunu çalıştırdığınızdan emin olun.")
    exit()

# 3. RASTGELE BİR HASTA SEÇ (Sağlamasını yapmak için)
rastgele_satir = df.sample(1).iloc[0]

# Seçilen hastanın verilerini al
gercek_dna = rastgele_satir['DNA_Dizisi']
gercek_prot = rastgele_satir['Protein_Dizisi']
gercek_durum = rastgele_satir['Gercek_Deger'] # 1 (Hasta) veya 0 (Sağlıklı)

# Sayısal verileri al: [Risk, MAF, Korunmusluk, Hidro, Polar, Mol_Agirlik]
gelen_veri = np.array([[
    rastgele_satir['Risk_Skoru'],
    rastgele_satir['MAF'],
    rastgele_satir['Korunmusluk'],
    rastgele_satir['Hidrofobiklik'],
    rastgele_satir['Polarite'],
    rastgele_satir['Mol_Agirlik']
]])

# 4. MODEL İÇİN HAZIRLA
seq_dna = pad_sequences(dna_tok.texts_to_sequences([gercek_dna]), maxlen=11, padding='post')
seq_prot = pad_sequences(prot_tok.texts_to_sequences([gercek_prot]), maxlen=11, padding='post')

# 5. MODELE SOR
tahmin_olasiligi = model.predict([seq_dna, seq_prot, gelen_veri])[0][0]
model_karari = 1 if tahmin_olasiligi > 0.5 else 0

# 6. KARŞILAŞTIRMA RAPORU
durum_metni = lambda x: "PATOJENİK (HASTA) ⚠️" if x == 1 else "BENIGN (SAĞLIKLI) ✅"

print("\n" + "="*50)
print(f"🧪 DOĞRULAMA TESTİ")
print("="*50)
print(f"Seçilen DNA      : {gercek_dna}")
print(f"Risk Skoru (Veri): {rastgele_satir['Risk_Skoru']:.4f}")
print("-" * 50)
print(f"GERÇEK DURUM     : {durum_metni(gercek_durum)}")
print(f"MODEL TAHMİNİ    : {durum_metni(model_karari)} (Olasılık: %{tahmin_olasiligi*100:.2f})")
print("-" * 50)

if gercek_durum == model_karari:
    print("SONUÇ: 🏆 MÜKEMMEL! Model doğru bildi.")
else:
    print("SONUÇ: ❌ HATA. Model bu zor örnekte yanıldı.")
print("="*50)

>> Model ve sözlükler yükleniyor...
>> Test sonuçları dosyası okunuyor...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step

🧪 DOĞRULAMA TESTİ
Seçilen DNA      : CCGCTAAATTA
Risk Skoru (Veri): 0.6052
--------------------------------------------------
GERÇEK DURUM     : PATOJENİK (HASTA) ⚠️
MODEL TAHMİNİ    : PATOJENİK (HASTA) ⚠️ (Olasılık: %62.92)
--------------------------------------------------
SONUÇ: 🏆 MÜKEMMEL! Model doğru bildi.
